#  4.1 Webscraping and scraping from PDF

In [14]:
import PyPDF2
import codecs
from urllib.request import urlopen
from bs4 import BeautifulSoup
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# 4.1.1 scraping from PDF

In [15]:
totalstr=''
for i in range(8):
    path='text{}.pdf'.format(i+1)
    test=PyPDF2.PdfFileReader(path)
    maxpage=test.numPages
    li=[]
    for i in range(maxpage):
        li.append(test.getPage(i))
    for i in range(maxpage):
        totalstr+=li[i].extractText()

# 4.1.2 web scraping

In [16]:
webpath=[]
webpath.append("https://www.sydney.com/things-to-do/adventure-and-sport/cycling")
webpath.append("https://www.campbelltown.nsw.gov.au/ServicesandFacilities/BicycleEducationCentre")
webpath.append("https://www.bikehiresydneyolympicpark.com.au/")
webpath.append("http://www.sydneycycleways.net/resources/sydney-rides/")
webpath.append("https://www.cityofsydney.nsw.gov.au/explore/getting-around/cycling")
webpath.append("https://www.cityofsydney.nsw.gov.au/explore/getting-around/cycling/dockless-bike-sharing")
webpath.append("https://whatson.cityofsydney.nsw.gov.au/events/free-bike-tune-ups-surry-hills")
webpath.append("https://concreteplayground.com/sydney/travel-leisure/leisure/the-ten-best-bike-rides-in-sydney")
webpath.append("http://sydneycyclingclub.org.au/tours/")
webpath.append("https://en.wikipedia.org/wiki/Cycling_in_Sydney")
webpath.append("http://www.sydneycyclist.com/")
webpath.append("https://www.mapmyride.com/au/sydney-new-south-wales/")
webpath.append("https://edition.cnn.com/travel/article/sydney-best-bike-paths/index.html")
webpath.append("https://wiki.openstreetmap.org/wiki/Sydney_Cycle_Routes")
webpath.append("https://www.bikenorth.org.au/")
webpath.append("https://theculturetrip.com/pacific/australia/articles/the-most-spectacular-cycling-routes-near-sydney-australia/")
st=''
for i in webpath:
    url =i
    handle_url = urlopen(url)
    bs = BeautifulSoup(handle_url, 'lxml')
    st+=bs.get_text()
totalstr+=st

# 4.1.3 Write to a file

In [17]:
text_file = codecs.open("Output.txt", "w","utf-8")
text_file.write(totalstr)
text_file.close()

# 4.2 Natural language processing

In [18]:
import os
from nltk.corpus import stopwords 
from math import sqrt
from math import ceil
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd

# 4.2.1 Read content from Output.txt

In [19]:
st=""
file=open("Output.txt","r")
file.seek(0,os.SEEK_END)
file_length=file.tell()
file.seek(0,os.SEEK_SET)
count=0
while count<file_length:
    try:
        st+=file.read(1)
    except:
        continue
    count+=1

# 4.2.2 text pre-processing

In [20]:
stop = set(stopwords.words('english')) 
filter_st= [word for word in st.split(' ') if word not in stop]
st=''
for i in filter_st:
    st+=i.replace("\n","")+" "

# 4.2.3 machine learning part

In [21]:
nlp = spacy.load('en_core_web_sm')
count=1
st_list=[]
if (len(st)>999999):
    count=ceil(len(st)/999999)
len_st=len(st)
for i in range(count):
    try:
        st_list.append(st[i*999999:(i+1)*999999])
    except:
        st_list.append(st[i*999999:len_st])
doc_list=[]
for i in range(count):
    doc_list.append(nlp(st_list[i]))
doc_ents=[]
for i in range(count):
    doc_ents+=list(doc_list[i].ents)

# 4.2.4 delete the word that must not be the name of distinct according to the machine learning result

In [22]:
maybe_placename=[]
for i in doc_ents:
    if i.label_ not in ['TIME','QUANTITY','DATE','LANGUAGE','MONEY']:
        maybe_placename.append(str(i))

# 4.2.5 import district name from csv

In [23]:
data=pd.read_csv("BikeSharingPods.csv")
area_name=list(data['name'])
district=[]
for i in area_name:
    to_add=i.split('-')[0]
    if to_add[-1]!=' ':
        district.append(to_add)
    else:
        district.append(to_add[:-1]) 

# 4.2.6 handle outlier and return the answer

In [24]:
dic={}
for i in district:
    count=0
    for ii in maybe_placename:
        if ii.lower() in i.lower():
            count+=1
    dic[i]=count
avg=0
for i in dic:
    avg+=dic[i]
avg=avg/len(dic)
for i in dic:
    if dic[i]>avg:
        dic[i]=int(sqrt(dic[i]))
print(dic)

{'Lavender Bay': 1, 'Enmore': 1, 'Redfern': 0, 'Kings Cross': 0, 'Newtown South': 1, 'Paddington': 0, 'Marrickville': 2, 'Zetland': 0, 'Elizabeth Bay': 1, 'St Leonards': 0, 'McMahons Point': 1, 'Glebe': 1, 'Summer Hill': 2, 'Haymarket': 5, 'Potts Point': 1, 'Randwick': 2, 'Clovelly': 0, 'Balmain': 1, 'Erskineville': 0, 'Surry Hills': 3, 'Darlinghurst': 0, 'Coogee': 0, 'Annandale': 0, 'Darlington': 0, 'Chippendale': 0, 'Woolloomooloo': 0, 'Petersham': 0, 'Manly': 0, 'North Sydney': 11, 'Newtown North': 5, 'Pyrmont': 2, 'Leichardt': 0, 'Neutral Bay': 1, 'Artarmon': 1, 'Fairlight': 7, 'Sydney CBD': 10, 'Newtown': 1, 'McMahons Point ': 1, 'Rozelle': 2, 'Ultimo': 0, 'Birchgrove': 0, 'Millers Point': 1, 'Bondi': 0, 'Bondi Junction': 1, 'Camperdown': 0, 'Bondi North': 4, 'Chatswood': 3, 'Milsons Point': 1, 'Mosman': 0, 'Bronte': 0, 'Dulwich Hill': 1, 'Waverton': 0, 'Forest Lodge': 1, 'Cremorne': 0, 'Balmain East': 2, 'Kirribilli': 1, 'Parramatta': 3, 'Lane Cove': 3, 'Kingsford': 0, 'Kensingto